In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, Batch
import numpy as np
import pandas as pd

In [2]:
jina_dim = 768
jina_dist = Distance.COSINE
jina_collection_name = "beer-jina"

client = QdrantClient(url="http://qdrant:6333", prefer_grpc=True)

try:
    client.create_collection(
        collection_name=jina_collection_name,
        vectors_config=VectorParams(size=jina_dim, distance=jina_dist),
    )
except Exception:
    pass

In [3]:
df = pd.read_json("./to-vec.json")
vec = np.load("./embedding-jina-id-3.npz").get("arr_0")
df["vector"] = vec.tolist()

In [4]:
df

,id,to_vectorize,vector
0,1,the beer Hocus Pocus from brewery Magic Hat (B...,"[-0.0039892346, -0.045099236, 0.050918385, 0.0..."
1,2,the beer Grimbergen Blonde from brewery Brouwe...,"[-0.020186651, -0.027529828, 0.011570356, 0.02..."
2,3,the beer Widdershins Barleywine from brewery L...,"[0.013534898, -0.02075706, 0.012209548, 0.0202..."
3,4,the beer Lucifer from brewery Brouwerij Liefma...,"[-0.015566914, -0.0016491069, 0.017164018, 0.0..."
4,5,the beer Bitter from brewery Ridgeway Brewing ...,"[0.008074477, -0.028539756, 0.033621587, 0.010..."
...,...,...,...
5896,5910,the beer Honey Rye Ale from brewery Abita Brew...,"[-0.025921917, -0.057531673, 0.08032945, 0.003..."
5897,5911,the beer Strawberry from brewery Abita Brewing...,"[-0.03335241, -0.061649293, 0.06562442, 0.0025..."
5898,5912,the beer Satsuma Wit from brewery Abita Brewin...,"[-0.010935032, -0.037077338, 0.07624469, 0.015..."
5899,5913,the beer Abhi beer from brewery Abhi Brewery (...,"[-0.0061397105, -0.036626168, 0.006759288, -0...."


In [5]:
def row_to_point(row: pd.Series) -> PointStruct:
    data = {
        "id": row.id,
        "vector": row.vector,
        "payload": {"description": row.to_vectorize}
    }
    return PointStruct(**data)

In [6]:
operation_info = client.upsert(
    collection_name=jina_collection_name,
    wait=True,
    points=df.apply(row_to_point, axis=1).to_list(),
)

print(operation_info)


operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [7]:
df.iloc[0].to_vectorize

'the beer Hocus Pocus from brewery Magic Hat (Burlington microbrewers of Humble Patience, Fat Angel, #9, Blind Faith IPA, and Heart of Darkness Oatmeal Stout.) crafts the beer Hocus Pocus defined as Our take on a classic summer ale.  A toast to weeds, rays, and summer haze.  A light, crisp ale for mowing lawns, hitting lazy fly balls, and communing with nature, Hocus Pocus is offered up as a summer sacrifice to clodless days.\r\n\r\nIts malty sweetness finishes tart and crisp and is best apprediated with a wedge of orange.. Spec of the beer are: ABV=4.5, IBU=0, SRM=0'

In [8]:
client.query_points(
    collection_name=jina_collection_name,
    query=df.iloc[0].vector,
    with_payload=True,
    limit=3
).points

[ScoredPoint(id=1, version=0, score=1.0, payload={'description': 'the beer Hocus Pocus from brewery Magic Hat (Burlington microbrewers of Humble Patience, Fat Angel, #9, Blind Faith IPA, and Heart of Darkness Oatmeal Stout.) crafts the beer Hocus Pocus defined as Our take on a classic summer ale.  A toast to weeds, rays, and summer haze.  A light, crisp ale for mowing lawns, hitting lazy fly balls, and communing with nature, Hocus Pocus is offered up as a summer sacrifice to clodless days.\r\n\r\nIts malty sweetness finishes tart and crisp and is best apprediated with a wedge of orange.. Spec of the beer are: ABV=4.5, IBU=0, SRM=0'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3949, version=0, score=0.8944739103317261, payload={'description': 'the beer HIPA from brewery Magic Hat (Burlington microbrewers of Humble Patience, Fat Angel, #9, Blind Faith IPA, and Heart of Darkness Oatmeal Stout.) crafts the beer HIPA defined as . Spec of the beer are: ABV=6.8, IBU=0, SR